<a href="https://colab.research.google.com/github/Magaton1010/image-analysis/blob/main/shadow_removall_novel_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Parameters You Can Adjust:
Threshold Values for Shadow Mask Creation:

threshold_s: Controls the sensitivity to low saturation values. Lower values will result in more areas being classified as shadow.
threshold_v: Controls the sensitivity to low brightness values. Lower values will result in more areas being classified as shadow.
Morphological Operations Kernel Size:

kernel_size: Controls the size of the kernel used in morphological operations. Larger kernels will result in more aggressive dilation and erosion, which can help refine the shadow mask but may also remove small details.
Low Saturation Threshold for Mask Creation:

Ls: Controls the sensitivity to low saturation regions for creating the mask. Lower values will classify more areas as low saturation.
How to Adjust the Parameters:
threshold_s and threshold_v: Increase or decrease these values to fine-tune which parts of the image are detected as shadows.
kernel_size: Change the kernel size to control the extent of the morphological operations, affecting the refinement of the shadow mask.
Ls: Adjust this value to control the sensitivity of detecting low saturation regions.

In [ ]:
import cv2
import numpy as np



#threshold_s and threshold_v: Increase or decrease these values to fine-tune which parts of the image are detected as shadows.
#kernel_size: Change the kernel size to control the extent of the morphological operations, affecting the refinement of the shadow mask.
#Ls: Adjust this value to control the sensitivity of detecting low saturation regions.


# Parameters to adjust
threshold_s = 100  # Saturation threshold for shadow mask creation
threshold_v = 100  # Value (brightness) threshold for shadow mask creation
kernel_size = 7   # Kernel size for morphological operations
Ls = 35           # Low saturation threshold for mask creation


# Step 1: Read input image
input_image_path = r'image_original.jpg'
output_dir = r'output_images.jpg'

image = cv2.imread(input_image_path)

# Ensure the output directory exists
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Step 2: Convert to HSV color space
image_hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
cv2.imwrite(f'{output_dir}image_hsv.jpg', image_hsv)

# Step 3: Create Shadow Mask
def create_shadow_mask(image_hsv, threshold_s=50, threshold_v=50):
    S = image_hsv[:, :, 1]  # Saturation channel
    V = image_hsv[:, :, 2]  # Value channel
    shadow_mask = (S < threshold_s) & (V < threshold_v)
    return shadow_mask.astype(np.uint8) * 255

shadow_mask = create_shadow_mask(image_hsv, threshold_s, threshold_v)
cv2.imwrite(f'{output_dir}shadow_mask.jpg', shadow_mask)

# Step 4: Refine Shadow Mask
kernel = np.ones((kernel_size, kernel_size), np.uint8)
shadow_mask_refined = cv2.morphologyEx(shadow_mask, cv2.MORPH_CLOSE, kernel)
shadow_mask_refined = cv2.morphologyEx(shadow_mask_refined, cv2.MORPH_OPEN, kernel)
cv2.imwrite(f'{output_dir}shadow_mask_refined.jpg', shadow_mask_refined)

# Step 5: Convert to CIE L*a*b Color Space
image_lab = cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
cv2.imwrite(f'{output_dir}image_lab.jpg', image_lab)

# Step 6: Thresholding and Mask Creation
def threshold_low_saturation_and_blue(image_hsv, shadow_mask, Ls=25):
    S = image_hsv[:, :, 1]  # Saturation channel
    B = image[:, :, 0]  # Blue channel in BGR image

    # Check where B channel is the maximum in the BGR image and S is below the threshold
    Lb = (B == np.max(image, axis=2)) & (S < Ls)

    Ks = Lb & (shadow_mask > 0)
    Ku = Lb & (shadow_mask == 0)

    return Lb, Ks, Ku

Lb, Ks, Ku = threshold_low_saturation_and_blue(image_hsv, shadow_mask_refined, Ls)
cv2.imwrite(f'{output_dir}Lb.jpg', Lb.astype(np.uint8) * 255)
cv2.imwrite(f'{output_dir}Ks.jpg', Ks.astype(np.uint8) * 255)
cv2.imwrite(f'{output_dir}Ku.jpg', Ku.astype(np.uint8) * 255)

# Step 7: Dilation of Shadow Mask
dilated_shadow_mask = cv2.dilate(shadow_mask_refined, kernel, iterations=1)
cv2.imwrite(f'{output_dir}dilated_shadow_mask.jpg', dilated_shadow_mask)

# Step 8: Local Color Transfer
def color_transfer(image_lab, shadow_mask, dilated_shadow_mask, Ks, Ku):
    output_image = image_lab.copy()

    for channel in range(3):  # L, a, b channels
        mean_shadow = cv2.mean(image_lab[:, :, channel], mask=shadow_mask)[0]
        mean_unshadow = cv2.mean(image_lab[:, :, channel], mask=(1 - shadow_mask))[0]

        std_shadow = np.std(image_lab[:, :, channel][shadow_mask > 0])
        std_unshadow = np.std(image_lab[:, :, channel][shadow_mask == 0])

        Ks_mask = Ks.astype(np.uint8)
        Ku_mask = Ku.astype(np.uint8)

        output_image[:, :, channel] = np.where(
            Ks_mask > 0,
            ((image_lab[:, :, channel] - mean_shadow) * (std_unshadow / std_shadow) + mean_unshadow),
            output_image[:, :, channel]
        )

    return output_image

corrected_image_lab = color_transfer(image_lab, shadow_mask_refined, dilated_shadow_mask, Ks, Ku)
corrected_image_bgr = cv2.cvtColor(corrected_image_lab, cv2.COLOR_Lab2BGR)
cv2.imwrite(f'{output_dir}corrected_image_lab.jpg', corrected_image_lab)
cv2.imwrite(f'{output_dir}corrected_image_bgr.jpg', corrected_image_bgr)

# Step 9: Color Tuning
def color_tuning(image_hsv, corrected_image_hsv):
    H, S, V = cv2.split(image_hsv)
    H_t, S_t, V_t = cv2.split(corrected_image_hsv)

    S_t = (S_t + S) // 2

    tuned_image_hsv = cv2.merge([H_t, S_t, V_t])
    return tuned_image_hsv

corrected_image_hsv = cv2.cvtColor(corrected_image_bgr, cv2.COLOR_BGR2HSV)
tuned_image_hsv = color_tuning(image_hsv, corrected_image_hsv)
final_image_bgr = cv2.cvtColor(tuned_image_hsv, cv2.COLOR_HSV2BGR)
cv2.imwrite(f'{output_dir}tuned_image_hsv.jpg', tuned_image_hsv)
cv2.imwrite(f'{output_dir}final_image_bgr.jpg', final_image_bgr)

print("Processing complete. Check the output_images directory for results.")


Initial Parameters:

Start with the default parameters provided in the code.
Visual Inspection:

Run the script and inspect the output images, especially the shadow mask (shadow_mask_refined.jpg) and the final corrected image (final_image_bgr.jpg).
Adjust Threshold Values:

If the shadow mask is too aggressive (includes non-shadow regions), increase threshold_s and threshold_v.
If the shadow mask is not capturing all shadows, decrease threshold_s and threshold_v.
Adjust Kernel Size:

If the shadow mask contains noise or small unwanted details, increase kernel_size to remove these artifacts.
If important details are being removed, decrease kernel_size.
Analyze Results:

Use the analyze_results function to print the mean and standard deviation of the pixel values in the shadow and corrected regions. Compare these statistics to see if the shadow removal is effective.
Iterate:

Repeat the process of adjusting one parameter at a time, re-running the script, and inspecting the results until you achieve satisfactory shadow removal.